In [1]:
import pandas as pd
import os

In [2]:
def PointID(bh):
     return "544WTG23-BH" + bh[3:5]

In [3]:
cone_df = pd.read_excel("data/cone area.xlsx")
cone_num = cone_df["Cone No."].to_list()
cone_area = cone_df["Cone Ratio"].to_list()
cone_dict = {str(num):area for num, area in zip(cone_num , cone_area)}

In [4]:
folder = "data/raw_CPT"
file_list = os.listdir(folder)
data = []
for filename in file_list:
    try:
        # Read file then close it
        file = open(f"{folder}/{filename}", "r")
        txt = file.readlines()
        file.close()
        # Extract data
        readings = txt[txt.index('#EOH=\n')+1:]     # find where "EOH" is to locate data
        cpt_hole = filename[filename.index("CPT"):].replace(".GEF", "")
        a_line = txt[txt.index("#MEASUREMENTTEXT= 3, Lease OCS-A 0544, name of location\n") + 1]
        a = a_line[a_line.index('.')+1 : a_line.index('cone type and serial')-2]
        txt = [[cpt_hole] + t.replace(" \n", "").split() + [cone_dict[a]] for t in readings]    # replace newline character
        data += txt
    except Exception as e:
        print(filename, e)

In [5]:
data_df = pd.DataFrame(data, columns=["CPT_hole", "Depth (m)", "qc (MPa)", "fs (MPa)", "Inc. NS", "Inc EW", "u2 (MPa)", "time (s)", "a"])
data_df

,CPT_hole,Depth (m),qc (MPa),fs (MPa),Inc. NS,Inc EW,u2 (MPa),time (s),a
0,CPT10,0.000000e+000,3.214457e-003,0.000000e+000,-4.251420e-001,3.727599e-001,5.972022e-004,-3.120190e+003,0.8
1,CPT10,1.000000e-002,3.214457e-003,0.000000e+000,-1.389042e+000,-6.968279e-001,5.972022e-004,-3.119190e+003,0.8
2,CPT10,2.000000e-002,3.214457e-003,0.000000e+000,-9.529045e-002,2.150538e-002,5.972022e-004,-3.118530e+003,0.8
3,CPT10,3.000000e-002,3.214457e-003,0.000000e+000,-1.979109e-001,8.817204e-001,5.972022e-004,-3.117880e+003,0.8
4,CPT10,4.000000e-002,3.214457e-003,0.000000e+000,-5.350925e-001,7.777778e-001,5.972022e-004,-3.117220e+003,0.8
...,...,...,...,...,...,...,...,...,...
46587,CPT11,1.461024e+001,4.534301e+001,0.000000e+000,1.638807e-001,-2.779262e+000,-6.816239e+000,6.575360e+003,0.8
46588,CPT11,1.462024e+001,4.555753e+001,0.000000e+000,1.757620e-001,-3.620167e+000,-1.474359e+000,6.575950e+003,0.8
46589,CPT11,1.463024e+001,5.176046e+001,0.000000e+000,2.298426e-001,-3.841083e+000,-2.784900e+000,6.576510e+003,0.8
46590,CPT11,1.464024e+001,3.767036e+001,0.000000e+000,1.884628e-001,-3.734188e+000,-1.509972e+000,6.577090e+003,0.8


In [6]:
data_df["PointID"] = data_df.apply(lambda x: PointID(x.CPT_hole), axis=1)
data_df = data_df[["PointID", "CPT_hole", "Depth (m)", "qc (MPa)", "fs (MPa)", "Inc. NS", "Inc EW", "u2 (MPa)", "time (s)"]]
data_df

,PointID,CPT_hole,Depth (m),qc (MPa),fs (MPa),Inc. NS,Inc EW,u2 (MPa),time (s)
0,544WTG23-BH10,CPT10,0.000000e+000,3.214457e-003,0.000000e+000,-4.251420e-001,3.727599e-001,5.972022e-004,-3.120190e+003
1,544WTG23-BH10,CPT10,1.000000e-002,3.214457e-003,0.000000e+000,-1.389042e+000,-6.968279e-001,5.972022e-004,-3.119190e+003
2,544WTG23-BH10,CPT10,2.000000e-002,3.214457e-003,0.000000e+000,-9.529045e-002,2.150538e-002,5.972022e-004,-3.118530e+003
3,544WTG23-BH10,CPT10,3.000000e-002,3.214457e-003,0.000000e+000,-1.979109e-001,8.817204e-001,5.972022e-004,-3.117880e+003
4,544WTG23-BH10,CPT10,4.000000e-002,3.214457e-003,0.000000e+000,-5.350925e-001,7.777778e-001,5.972022e-004,-3.117220e+003
...,...,...,...,...,...,...,...,...,...
46587,544WTG23-BH11,CPT11,1.461024e+001,4.534301e+001,0.000000e+000,1.638807e-001,-2.779262e+000,-6.816239e+000,6.575360e+003
46588,544WTG23-BH11,CPT11,1.462024e+001,4.555753e+001,0.000000e+000,1.757620e-001,-3.620167e+000,-1.474359e+000,6.575950e+003
46589,544WTG23-BH11,CPT11,1.463024e+001,5.176046e+001,0.000000e+000,2.298426e-001,-3.841083e+000,-2.784900e+000,6.576510e+003
46590,544WTG23-BH11,CPT11,1.464024e+001,3.767036e+001,0.000000e+000,1.884628e-001,-3.734188e+000,-1.509972e+000,6.577090e+003


In [7]:
data_df.to_excel("data/CPT data_raw.xlsx")